In [1]:
%load_ext autoreload
%autoreload 2

from experiments.all import train_and_evaluate
from tsm.configuration import TSMConfigParser
import docker
import logging

logging.basicConfig(level=logging.INFO)

# load configuration
config = TSMConfigParser()
config.read("/home/pablo/tesis/tsm-pipeline/code/configs/pablo-bestia.cfg")
config.check()
docker_client = docker.from_env()

loading the dataset and creating a random state to make everything repeatable

In [2]:
from scripts.docker import read_dbs_dataset
from numpy.random import RandomState

xss_dbs = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/datasets/xss_1.txt")
# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

print(len(xss_dbs))

90


smoke testing run to check everything is working

In [7]:
import tempfile

test_and_train = [
    xss_dbs[0],
]

res_dir = tempfile.mkdtemp()
print("using %s for smoke run" % (res_dir))
# !rm -rf $res_dir/import pandas as pd

concated_results = pd.concat(collected_results)
pd.concat([concated_results, concated_results.apply(['mean'])])*
import pandas as pd

concated_results = pd.concat(collected_results)
pd.concat([concated_results, concated_results.apply(['mean'])])
train_and_evaluate(
    config=config,
    results_dir=res_dir,
    query_type="xss",
    train=test_and_train,
    test=test_and_train,
    docker_client=docker_client,
    skip=["train", "evalute_worse", "evaluate_boosted", "evaluate_v0"]
)

INFO:experiments.all:train_and_evaluate with train size 1, test size 1
INFO:experiments.all:Running worse evaluation
INFO:scripts.evaluate:creating /tmp/tmp6ddjrlqd/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes


using /tmp/tmp6ddjrlqd for smoke run


INFO:scripts.evaluate:Evaluating Breeze - breeze.js.samples
/home/pablo/tesis/tsm-pipeline/code/env/lib/python3.10/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/pablo/tesis/tsm-pipeline/code/env/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.0,0.0,NaN,0,0,0,0,0


as a second sanity, doing a run with 10/10 over the first 20 sample

In [4]:
from random import sample
from sklearn.model_selection import train_test_split

sample_20 = sample(xss_dbs, 20)
train, test = train_test_split(sample_20, train_size=.5, shuffle=True, random_state=rand)
print("train %d, test %d" %(len(train), len(test)))

train 10, test 10


In [ ]:
res_dir = tempfile.mkdtemp()
print("using %s for 20 run" % (res_dir))

train_and_evaluate(
    config=config,
    results_dir=res_dir,
    query_type="xss",
    train=train,
    test=test,
    docker_client=docker_client,
)

igual que en path, la idea es hacer un corrida haciendo un k-fold pequeño para corrorar que el problema es el dataset

In [4]:
from random import sample
from sklearn.model_selection import KFold
import numpy as np

sample_30 = sample(xss_dbs, 30)
print("using sample of %d" %(len(sample_30)))
# fixing random state to make this repeatable
kf = KFold(n_splits=3)
collected_results = []
base_results_dir = "/home/pablo/dev-results/xssrun2-30-3fold-"
for i, (train_index, test_index) in enumerate(kf.split(sample_30)):
    print("Fold %d" % (i))
    res_dir = "%s%d" % (base_results_dir, i)
    train_split = np.take(sample_30, train_index)
    test_split = np.take(sample_30, test_index)

    scores = train_and_evaluate(
        config=config,
        results_dir=res_dir,
        query_type="xss",
        train=train_split.tolist(),
        test=test_split.tolist(),
        docker_client=docker_client,
    )
    print(scores)
    collected_results.append(scores)
print("done")

INFO:experiments.all:train_and_evaluate with train size 20, test size 10
INFO:experiments.all:Running training
INFO:scripts.docker:creating /home/pablo/dev-results/xssrun2-30-3fold-0 dir. Omitting error if already existing
INFO:scripts.docker:creating /tmp/log dir. Omitting error if already existing


using sample of 30
Fold 0


INFO:scripts.docker:running at container 4fc0d5f34d2e834fa5345eb0a49f86f4631c82d48ca994297262a0f43e32129a. Use `docker logs 4fc0d5f34d2e834fa5345eb0a49f86f4631c82d48ca994297262a0f43e32129a --tail 10 --follow` to follow progress
INFO:scripts.docker:running combine scores
INFO:misc.combinescores:globbing folder: /home/pablo/dev-results/xssrun2-30-3fold-0/*/reprScores.txt
INFO:misc.combinescores:working on 10 reprScores files
INFO:experiments.all:Running worse evaluation
INFO:scripts.evaluate:creating /home/pablo/dev-results/xssrun2-30-3fold-0/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating Soontao - openui5
INFO:scripts.evaluate:Evaluating RetireJS - retire.js
INFO:scripts.evaluate:Evaluating HelloZeroNet - ZeroNet
INFO:scripts.evaluate:Evaluating DimiMikadze - laravel-angular-cms
INFO:scripts.evaluate:Evaluating Xowap - je-vous-presente-bernard
INFO:scripts.evaluate:Evaluating EPCCed - EPCC-Facilit

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                       18                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                0                  3               6.0  
Fold 1


INFO:scripts.docker:running at container c6f7e2944ef57475eb6d239105f6a7936ed5658f6307bf2e529a23c655a0d69d. Use `docker logs c6f7e2944ef57475eb6d239105f6a7936ed5658f6307bf2e529a23c655a0d69d --tail 10 --follow` to follow progress
INFO:scripts.docker:running combine scores
INFO:misc.combinescores:globbing folder: /home/pablo/dev-results/xssrun2-30-3fold-1/*/reprScores.txt
INFO:misc.combinescores:working on 15 reprScores files
INFO:experiments.all:Running worse evaluation
INFO:scripts.evaluate:creating /home/pablo/dev-results/xssrun2-30-3fold-1/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating benas - gamehub.io
INFO:scripts.evaluate:Evaluating c2corg - tpb
INFO:scripts.evaluate:Evaluating Fuzzyma - jquery.dataFilter
INFO:scripts.evaluate:Evaluating FirebaseExtended - firepad
INFO:scripts.evaluate:Evaluating Esri - storymap-cascade
INFO:scripts.evaluate:Evaluating OanaMariaRosu - feedback
INFO:scripts.e

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       NaN                        0                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                0                  0                 0  
Fold 2


INFO:scripts.docker:running at container a4d5bfa71c4d43e520b12e3b13ddfca5bb00b858148f223b846098b7e62426bd. Use `docker logs a4d5bfa71c4d43e520b12e3b13ddfca5bb00b858148f223b846098b7e62426bd --tail 10 --follow` to follow progress
INFO:scripts.docker:running combine scores
INFO:misc.combinescores:globbing folder: /home/pablo/dev-results/xssrun2-30-3fold-2/*/reprScores.txt
INFO:misc.combinescores:working on 11 reprScores files
INFO:experiments.all:Running worse evaluation
INFO:scripts.evaluate:creating /home/pablo/dev-results/xssrun2-30-3fold-2/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating Matttaylor8910 - strategery
INFO:scripts.evaluate:Evaluating bpm - bpm
INFO:scripts.evaluate:Evaluating Raxa - Raxa-JSS
INFO:scripts.evaluate:Evaluating ProjectTLU17 - Study
INFO:scripts.evaluate:Evaluating ClojureBridge - curriculum
INFO:scripts.evaluate:Evaluating FullstackAcademy - Study-Saturday-Fullstack
INFO

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                        8                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                0                  2               4.0  
done


In [5]:
import pandas as pd

concated_results = pd.concat(collected_results)
pd.concat([concated_results, concated_results.apply(['mean'])])

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.0,0.0,0.0,18.000000,0.0,0.0,3.000000,6.000000
0,0.0,0.0,NaN,0.000000,0.0,0.0,0.000000,0.000000
0,0.0,0.0,0.0,8.000000,0.0,0.0,2.000000,4.000000
mean,0.0,0.0,0.0,8.666667,0.0,0.0,1.666667,3.333333
